In [13]:

# This notebook is used to generate triplet of music attributes (melody) for lyrics in the test data.


In [12]:

import numpy as np
import tensorflow as tf
import midi_statistics
import utils
import pickle
import matplotlib.pyplot as plt


In [7]:

test_data = np.load('./data/processed_dataset_matrices/test_data_matrix.npy')

test_data.shape


(1051, 460)

In [4]:

generated_songs = []

with tf.Session(graph=tf.Graph()) as sess:

    model_path = 'saved_gan_models/saved_model_best_overall_mmd'

    tf.saved_model.loader.load(sess, [], model_path)
    graph = tf.get_default_graph()
    keep_prob = graph.get_tensor_by_name("model/keep_prob:0")
    input_metadata = graph.get_tensor_by_name("model/input_metadata:0")
    input_songdata = graph.get_tensor_by_name("model/input_data:0")
    output_midi = graph.get_tensor_by_name("output_midi:0")
    feed_dict = {}
    feed_dict[keep_prob.name] = 1
    condition = []
    feed_dict[input_metadata.name] = condition
    
    for i in range(len(test_data)):
        feed_dict[input_songdata.name] = np.random.uniform(size=(1, 20, 3))
        condition.append(np.split(test_data[i][3 * 20:], 20))
        feed_dict[input_metadata.name] = condition
        generated_features = sess.run(output_midi, feed_dict)
        sample = [x[0, :] for x in generated_features]
        sample = midi_statistics.tune_song(utils.discretize(sample))
        generated_songs.append(sample)
        condition = []


Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.
INFO:tensorflow:Restoring parameters from saved_gan_models/saved_model_best_overall_mmd/variables/variables


In [8]:

generated_songs = np.asarray(generated_songs)
generated_songs.shape


(1051, 20, 3)

In [9]:

generated_pitches = generated_songs[:, :, 0]
generated_lengths = generated_songs[:, :, 1]
generated_rests   = generated_songs[:, :, 2]

generated_pitches.shape, generated_lengths.shape, generated_rests.shape


((1051, 20), (1051, 20), (1051, 20))

In [10]:

np.save(model_path + '/generated_songs',   generated_songs)
np.save(model_path + '/generated_pitches', generated_pitches)
np.save(model_path + '/generated_lengths', generated_lengths)
np.save(model_path + '/generated_rests',   generated_rests)
